In [6]:
import torch
import torch.nn as nn
import numpy as np

time_steps = 2
batch_size = 2 # this is # files you're loading
input_size = 3 # this is # of features extracted
hidden_size = 10
nclasses = 5

# define model class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # "nn.Linear" performs the last step of prediction, where it maps the hidden layer to # classes
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# set up model
model = RNN(input_size, hidden_size, 1, nclasses)
# defines optimization function (this is using stochastic gradient descent)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# defines loss function
criterion = nn.CrossEntropyLoss()

# say your input array looks like this (represents 2 files, each of which stores 3 features over 2 timesteps:
trainX = [
    [[1, 2, 3], [3, 2, 1]],
    [[1, 3, 2], [0, 3, 3]]
]

# say the true emotion classes are this
trainY = [1, 3]

testX = [[[2, 2, 2], [3, 0, 1]]]
testY = [4]

# Train model
print("Training")
for epoch in range(10):
    total_loss = 0
    correct = 0
    for i in range(len(trainX)):
        x = torch.tensor(trainX[i], dtype=torch.float).unsqueeze(0)
        y = torch.tensor(trainY[i]).unsqueeze(0)
        # Step 3. Run our forward pass.
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        if predicted == y:
            correct += 1

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = criterion(outputs, y)
        total_loss += loss.item()

        # clear gradient for next step of training
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    total_loss /= len(trainX)
    correct /= len(trainX)
    print("Epoch #%d, Loss: %f, Accuracy: %f" % (epoch, total_loss, correct))

# Test model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(testX)):
        x = torch.tensor(testX[i], dtype=torch.float).unsqueeze(0)
        y = torch.tensor(testY[i]).unsqueeze(0)

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        if predicted == y:
            correct += 1

    print('Test Accuracy of the model: {} %'.format(100 * correct / total)) 


Training
Epoch #0, Loss: 1.826429, Accuracy: 0.000000
Epoch #1, Loss: 1.683319, Accuracy: 0.000000
Epoch #2, Loss: 1.560792, Accuracy: 0.500000
Epoch #3, Loss: 1.454891, Accuracy: 0.500000
Epoch #4, Loss: 1.361935, Accuracy: 0.500000
Epoch #5, Loss: 1.278868, Accuracy: 0.500000
Epoch #6, Loss: 1.203380, Accuracy: 0.500000
Epoch #7, Loss: 1.133781, Accuracy: 0.500000
Epoch #8, Loss: 1.068804, Accuracy: 0.500000
Epoch #9, Loss: 1.007437, Accuracy: 1.000000
Test Accuracy of the model: 0.0 %
